In [ ]:
%tensorflow_version 1.x

In [ ]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

In [ ]:
print(tf.__version__)

1.15.2


In [ ]:
!git clone https://github.com/hiddenmaze/InteractivePickup.git InteractivePickup --origin upstream

Cloning into 'InteractivePickup'...
remote: Enumerating objects: 5281, done.
remote: Total 5281 (delta 0), reused 0 (delta 0), pack-reused 5281
Receiving objects: 100% (5281/5281), 221.05 MiB | 36.86 MiB/s, done.
Resolving deltas: 100% (2788/2788), done.
Checking out files: 100% (5468/5468), done.


In [ ]:
import csv
import pandas as pd
contents = []

with open('/content/drive/MyDrive/machine_listening_2020/Project/train_model/speech_word2vec.txt', 'r') as f:
    vec_list = []
    for row in f:
        row2 = row.split()
        vec_list.append(row2)

In [ ]:
vec = pd.DataFrame(vec_list)
vec = vec.set_index(0).T.to_dict('list')

In [ ]:
%cd /content/InteractivePickup

import numpy as np
from os import listdir, remove
from os.path import isfile, join
from scipy import misc as misc
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal as mulnorm
from utils import load_w2v

### Import word2vec model
def preprocess():
 #   w2v_path = './data/GoogleNews-vectors-negative300.bin'
 #   w2v_model = load_w2v(w2v_path)

 #   dim_embed = w2v_model['woman'].shape[0]

    ### Calculate Maximum length of the language command
    script_path = '/content/InteractivePickup/data/train_script'
    files = [f for f in listdir(script_path) if isfile(join(script_path, f))]

    total_words = set()
    max_len = 0
    num_data = 0
    dim_embed = 100

    for tmp_file in files:
        curr_file = open(join(script_path, tmp_file), 'r')
        curr_lines = curr_file.readlines()
        for line in curr_lines:
            words = line.split()
            for word in words:
                if words.index(word) > 1:
                    total_words.add(word)
            if max_len < len(words)-2:
                max_len = len(words)-2
        num_data += len(curr_lines)

    total_words = list(total_words)

    max_len = max_len + 5

    print('Total data number: %d' % num_data)

    ### Set Empty data arrays
    inputs = np.zeros((num_data, dim_embed, max_len))
    outputs = np.zeros((num_data, 2))
    img_idx = np.zeros((num_data, 1))
    seq_len = np.zeros((num_data, 1))
    tmp_num = 0

    ### Start Preprocess : delete # for re-generate heatmaps
    for file_idx in (files):
        curr_file = open(join(script_path, file_idx), 'r')
        curr_lines = curr_file.readlines()

        prev_output = [0, 0]
        for line in curr_lines:
            words = line.split()
            for i, word in enumerate(words):
                if i <= 1:
                    outputs[tmp_num, i] = float(word)
                else:
                    if word not in vec.keys():
                        inputs[tmp_num, :, i-2] = np.zeros((300,))
                    else:
                        inputs[tmp_num, :, i-2] = vec[word]
            img_idx[tmp_num, 0] = float(file_idx[0:4])
            seq_len[tmp_num, 0] = len(words)-2

            '''
            if prev_output[0] != outputs[tmp_num, 0] or prev_output[1] != outputs[tmp_num, 1]:
                tmp_heatmap = np.zeros((250, 250))
                tmp_output = outputs[tmp_num, :].astype(int)
                print 'generate heatmap at %03d, %03d' % (tmp_output[0], tmp_output[1])
                tmp_cov = [[500, 0], [0, 500]]
                mvn = mulnorm([tmp_output[1], tmp_output[0]], tmp_cov)
                for k in range(tmp_heatmap.shape[0]):
                    for kk in range(tmp_heatmap.shape[1]):
                        tmp_heatmap[k, kk] = mvn.pdf([k, kk])
                tmp_heatmap = tmp_heatmap / np.max(tmp_heatmap)
                np.savez(('./data/train_heatmap/%s_%03d_%03d.npz')%(file_idx[0:4], tmp_output[0], tmp_output[1] ), tmp_heatmap)
            prev_output = outputs[tmp_num, :]
            '''        
            tmp_num += 1      

    ### Save preprocessed data
    np.savez('./data/preprocessed4HGN.npz', img_idx, seq_len, inputs, outputs, total_words)

/content/InteractivePickup


In [ ]:
preprocess()

In [ ]:
npzfile = np.load('./data/preprocessed4HGN.npz')

img_idx = npzfile['arr_0']
sen_len = npzfile['arr_1']
text_inputs = npzfile['arr_2']
pos_outputs = npzfile['arr_3']

In [ ]:
import numpy as np
import random
from skimage import io
from skimage.transform import resize
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable
from os import listdir, remove
from os.path import isfile, join
from google_drive_downloader import GoogleDriveDownloader as gdd
import gzip
import gensim

def load_w2v(w2v_path):
    if isfile(w2v_path) == False:
        print("Start downloading Google Word2Vec data")
        gdd.download_file_from_google_drive(file_id='0B7XkCwpI5KDYNlNUTTlSS21pQmM',
                                            dest_path=w2v_path+'.gz',
                                            unzip=False)
        inF = gzip.open(w2v_path+'.gz', 'rb')
        outF = open(w2v_path, 'wb')
        outF.write( inF.read() )
        inF.close()
        outF.close()

        remove(w2v_path+'.gz')
    print("Start loading Google Word2Vec data")
    w2v_model = gensim.models.KeyedVectors.load_word2vec_format(w2v_path, binary=True)
    print("Finished loading Google Word2Vec data")

    return w2v_model

def QGN_organize_data(img_path):
    img_resize = 64
    img_files = [f for f in listdir(img_path) if isfile(join(img_path, f))]

    total_images = dict()
    for f in img_files:
        tmp_img = io.imread(join(img_path, f))
        tmp_img = resize(tmp_img, [img_resize, img_resize], preserve_range=True)
        tmp_img = tmp_img / 255.0
        total_images[f[0:4]] = tmp_img
    
    return total_images
    
def HGN_organize_data(img_path, script_path):
    img_resize = 256
    heatmap_resize = 64
    
    img_files = [f for f in listdir(img_path) if isfile(join(img_path, f))]
    script_files = [f for f in listdir(script_path) if isfile(join(script_path, f))]

    total_images = dict()
    for f in img_files:
        tmp_img = io.imread(('%s/%s') % (img_path, f))
        tmp_img = resize(tmp_img, [img_resize, img_resize], preserve_range=True)
        tmp_img = tmp_img / 255.0
        total_images[f[0:4]] = tmp_img

    total_heatmaps = dict()
    for f in script_files:
        npzfile = np.load(('%s/%s') % (script_path, f))
        tmp_heatmap = npzfile['arr_0']
        tmp_heatmap = resize(tmp_heatmap, [heatmap_resize, heatmap_resize], preserve_range=True)
        total_heatmaps[f[0:12]] = tmp_heatmap
    return total_images, total_heatmaps
  
def divide_img_idx(img_idx, num_data):
    if isfile('./data/divide_img_idx.npz') == False:
        random_seed=1
        divide_rate=20

        tmp_img_idx = list(range(0,478))
        random.seed(random_seed)
        random.shuffle(tmp_img_idx)

        num_test_img = int((max(img_idx)+1)/divide_rate)
        test_img_idx = tmp_img_idx[0:num_test_img]
        train_img_idx = tmp_img_idx[num_test_img:len(tmp_img_idx)]

        np.savez('./data/divide_img_idx.npz', train_img_idx, test_img_idx)
    else:
        npzfile = np.load('./data/divide_img_idx.npz')
        train_img_idx = npzfile['arr_0']
        test_img_idx = npzfile['arr_1']
                               
    print('Test image index : %s' % (test_img_idx))

    return test_img_idx, train_img_idx
    
def get_test_idx(img_idx, num_data):
    test_img_idx, train_img_idx = divide_img_idx(img_idx, num_data)

    idx_whole = range(num_data)
    idx_test = []
    for i in test_img_idx:
        idx_test += np.where(img_idx == i)[0].tolist()
    idx_train = [i for i in idx_whole if i not in idx_test]
    
    return idx_test, idx_train

def QGN_divide_train_test(img_idx, sen_len, question_embeds, esti_maps, uncertainty_maps, answer_labels, num_data):
    idx_test, idx_train = get_test_idx(img_idx, num_data)

    num_train = len(idx_train)
    train_img_idx = img_idx[idx_train, :]
    train_sen_len = sen_len[idx_train, :]
    train_embeds = question_embeds[idx_train, :, :]
    train_esti = esti_maps[idx_train, :, :]
    train_uncertainty = uncertainty_maps[idx_train, :, :]
    train_answer_labels = answer_labels[idx_train, :]
    
    num_test = len(idx_test)
    test_img_idx = img_idx[idx_test, :]
    test_sen_len = sen_len[idx_test, :]
    test_embeds = question_embeds[idx_test, :, :]
    test_esti = esti_maps[idx_test, :, :]
    test_uncertainty = uncertainty_maps[idx_test, :, :]
    test_answer_labels = answer_labels[idx_test, :]

    print('Ended divided training and test dataset. Training : %d, Test : %d' % (num_train, num_test))                       
    
    return idx_train, idx_test, num_train, num_test,\
train_img_idx, train_sen_len, train_embeds,\
train_esti, train_uncertainty, train_answer_labels,\
test_img_idx, test_sen_len, test_embeds, test_esti, test_uncertainty, test_answer_labels
            
def HGN_divide_train_test(img_idx, sen_len, text_inputs, pos_outputs, num_data):
    idx_test, idx_train = get_test_idx(img_idx, num_data)

    num_train = len(idx_train)
    train_img_idx = img_idx[idx_train, :]
    train_sen_len = sen_len[idx_train, :]
    train_text_inputs = text_inputs[idx_train, :, :]
    train_pos_outputs = pos_outputs[idx_train, :]
    
    num_test = len(idx_test)    
    test_img_idx = img_idx[idx_test, :]
    test_sen_len = sen_len[idx_test, :]
    test_text_inputs = text_inputs[idx_test, :, :]
    test_pos_outputs = pos_outputs[idx_test, :]
    print('Ended divided training and test dataset. Training : %d, Test : %d' % (num_train, num_test))
    
    return idx_train, idx_test, num_train, num_test, \
           train_img_idx, train_sen_len, train_text_inputs, train_pos_outputs, \
           test_img_idx, test_sen_len, test_text_inputs, test_pos_outputs,
           

def load_test_img(img_path, curr_test_img_idx, img_resize, plot_flag):
    curr_test_img = np.zeros((1, img_resize, img_resize, 3))
    tmp_img = io.imread((img_path + '/%04d.jpg') % curr_test_img_idx)
    tmp_img = resize(tmp_img, [img_resize, img_resize], preserve_range=True)
    tmp_img = tmp_img / 255.0
    curr_test_img[0, :, :, :] = tmp_img

    if plot_flag == 1:
        plt.imshow(tmp_img)
        plt.show()
    
    return curr_test_img
    
def load_test_script(curr_test_input, w2v_model, dim_sentence, max_step_sentence):
    curr_embed_input = np.zeros((1, dim_sentence, max_step_sentence))
    curr_seq_len = np.zeros((1, 1))
    curr_words = curr_test_input.split()
    for i, word in enumerate(curr_words):
        if word not in w2v_model.vocab.keys():
            curr_embed_input[0, :, i] = np.zeros((300,))
        else:
            curr_embed_input[0, :, i] = w2v_model[word]
    curr_seq_len[0, 0] = len(curr_words)

    print("Ready the test input script")
    
    return curr_embed_input, curr_seq_len

def plot_HGN_result(curr_test_input, curr_test_img, mean_of_esti, uncertainty, bound_u=0.0, bound_c=0.0):
    fig = plt.figure(figsize=(10, 10))
    fig.subplots_adjust(hspace=0.2, wspace=0.3)
    fig.suptitle('Input script: ' + curr_test_input, size=30)

    ax1 = fig.add_subplot(2, 2, 1)
    ax1.imshow(resize(curr_test_img[0, :, :, :], [256, 256], preserve_range=True))
    ax1.set_title('input image', size=20)
    ax1.set_axis_off()

    ax2 = fig.add_subplot(2, 2, 2)
    plot2 = ax2.imshow(resize(mean_of_esti, [256, 256], preserve_range=True), cmap='jet')
    ax2.set_title('estimation map', size=20)
    ax2.set_axis_off()
    divider2 = make_axes_locatable(ax2)
    cax2 = divider2.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(plot2, ax=ax2, cax=cax2)

    ax3 = fig.add_subplot(2, 2, 3)
    if bound_u > 0:
        norm3 = mpl.colors.Normalize(vmin=0, vmax=bound_u)
        plot3 = ax3.imshow(resize(uncertainty, [256, 256], preserve_range=True), cmap='jet',
                          norm = norm3)
    else:
        norm3 = mpl.colors.Normalize(vmin=0, vmax=np.max(uncertainty))
        plot3 = ax3.imshow(resize(uncertainty, [256, 256], preserve_range=True), cmap='jet',
                          norm = norm3)
    ax3.set_title('uncertainty map', size=20)
    ax3.set_axis_off()
    divider3 = make_axes_locatable(ax3)
    cax3 = divider3.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(plot3, ax=ax3, cax=cax3)   
    
    ax4 = fig.add_subplot(2, 2, 4)
    if bound_c > 0:
        norm4 = mpl.colors.Normalize(vmin=0, vmax=bound_c)
        plot4 = ax4.imshow(resize(2*uncertainty + mean_of_esti, [256, 256], preserve_range=True), cmap='jet', norm=norm4)
    else:
        norm4 = mpl.colors.Normalize(vmin=0, vmax=np.max(2*uncertainty + mean_of_esti))
        plot4 = ax4.imshow(resize(2*uncertainty + mean_of_esti, [256, 256], preserve_range=True), cmap='jet', norm=norm4) 
    ax4.set_title('confidence map', size=20)
    ax4.set_axis_off()
    divider4 = make_axes_locatable(ax4)
    cax4 = divider4.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(plot4, ax=ax4, cax=cax4)

In [ ]:
from os.path import isfile, join
import numpy as np
from code_HGN_train import train
from preprocess4HGN import preprocess

num_data = img_idx.shape[0]
dim_sentence = text_inputs.shape[1]
max_step_sentence = text_inputs.shape[2]

img_path = './data/train_img'
script_path = './data/train_heatmap'

total_images, total_heatmaps = HGN_organize_data(img_path, script_path);

idx_train, idx_test, num_train, num_test, \
train_img_idx, train_sen_len, train_text_inputs, train_pos_outputs, \
test_img_idx, test_sen_len, test_text_inputs, test_pos_outputs = \
HGN_divide_train_test(img_idx, sen_len, text_inputs, pos_outputs, num_data=num_data)

Test image index : [154, 47, 59, 31, 419, 446, 217, 63, 152, 236, 289, 229, 434, 67, 45, 316, 352, 135, 285, 77, 318, 41, 142]
Ended divided training and test dataset. Training : 19287, Test : 1062


In [ ]:
import tensorflow as tf
from residual import Residual
from tensorflow.python.ops import variable_scope as vs

stddev = 0.01

def hourglass_with_rnn(curr_input, rnn_feat, numDepth, numIn, numOut, dr_rate):
    up1 = curr_input
    up1 = Residual(up1, numIn, numOut, dr_rate)

    low1 = tf.layers.max_pooling2d(curr_input, 2, 2)
    low1 = Residual(low1, numIn, numOut, dr_rate)
    
    if numDepth > 1:
        low2 = hourglass_with_rnn(low1, rnn_feat, numDepth-1, numIn, numOut, dr_rate)
    else:
        low2 = Residual(low1, numIn, numOut, dr_rate)
    
    low3 = Residual(low2, numIn, numOut-1, dr_rate)
    
    with tf.variable_scope("word_weights", reuse=True):
        if numDepth == 4:
            _W_o4 = tf.get_variable('W_o4')
            _b_o4 = tf.get_variable('b_o4')
        elif numDepth == 3:
            _W_o3 = tf.get_variable('W_o3')
            _b_o3 = tf.get_variable('b_o3')
        elif numDepth == 2:
            _W_o2 = tf.get_variable('W_o2')
            _b_o2 = tf.get_variable('b_o2')
        elif numDepth == 1:
            _W_o1 = tf.get_variable('W_o1')
            _b_o1 = tf.get_variable('b_o1')
        else:
            raise ValueError('Code need to be fixed due to changed numDepth')
    
    if numDepth == 4:
        fitted_rnn = tf.matmul(rnn_feat, _W_o4)+_b_o4
    elif numDepth == 3:
        fitted_rnn = tf.matmul(rnn_feat, _W_o3)+_b_o3
    elif numDepth == 2:
        fitted_rnn = tf.matmul(rnn_feat, _W_o2)+_b_o2
    elif numDepth == 1:
        fitted_rnn = tf.matmul(rnn_feat, _W_o1)+_b_o1
    else:
        raise ValueError('Code need to be fixed due to changed numDepth')

    fitted_rnn = tf.nn.dropout(fitted_rnn, 1-dr_rate)
    
    fitted_rnn = tf.reshape(fitted_rnn, [tf.shape(rnn_feat)[0], 
                                        tf.to_int32(tf.sqrt(tf.to_float(tf.shape(fitted_rnn)[1]))),
                                        tf.to_int32(tf.sqrt(tf.to_float(tf.shape(fitted_rnn)[1])))])
    fitted_rnn = tf.expand_dims(fitted_rnn, axis=3)
    
    fuse_low3 = tf.concat([low3, fitted_rnn], axis=3)
        
    up2 = tf.image.resize_nearest_neighbor(fuse_low3, 2*tf.shape(fuse_low3)[1:3])

    return tf.add(up1, up2)


def lin(curr_input, numIn, numOut, dr_rate):
    l = tf.layers.conv2d(curr_input, numOut, 1, padding='Same')
    l = tf.layers.dropout(l, rate=dr_rate)
    l = tf.layers.batch_normalization(l)
    return tf.nn.relu(l)


def createModel(curr_img, curr_sen, curr_sen_len,
                img_size, dim_sentence, max_step_sentence, 
                num_hg_Depth, dim_hg_feat, dim_rnn_cell, dim_output,
                dr_rate):

    # image size must be 256 by 256.
    curr_rnn_encoder= rnn_encoder(img_size, num_hg_Depth, dim_sentence, max_step_sentence, dim_rnn_cell, dr_rate)
    
    rnn_feat = curr_rnn_encoder.encode(curr_sen, curr_sen_len)
    
    with vs.variable_scope('HGN'):
        with vs.variable_scope('pre'):
            cnv1 = tf.layers.conv2d(curr_img, filters=dim_hg_feat/4, kernel_size=7, strides=2, padding='Same')
            cnv1 = tf.layers.dropout(cnv1, rate=dr_rate)

            cnv1 = tf.layers.batch_normalization(cnv1)
            cnv1 = tf.nn.relu(cnv1)
        
        with vs.variable_scope('r1'):
            r1 = Residual(cnv1, dim_hg_feat/4, dim_hg_feat/2, dr_rate)

        pool = tf.layers.max_pooling2d(r1, 2, 2)

        with vs.variable_scope('r4'):    
            r4 = Residual(pool, dim_hg_feat/2, dim_hg_feat/2, dr_rate)

        with vs.variable_scope('r5'):    
            r5 = Residual(r4, dim_hg_feat/2, dim_hg_feat, dr_rate)    
    
        with vs.variable_scope('hg'):  
            hg = hourglass_with_rnn(r5, rnn_feat, num_hg_Depth, dim_hg_feat, dim_hg_feat, dr_rate)

        with vs.variable_scope('ll'):      
            ll = Residual(hg, dim_hg_feat, dim_hg_feat, dr_rate)
            ll = lin(ll, dim_hg_feat, dim_hg_feat, dr_rate)
            
        with vs.variable_scope('out'): 
            Out = tf.layers.conv2d(ll, filters=dim_output, kernel_size=1, strides=1, padding='Same')
            Out = tf.layers.dropout(Out, rate=dr_rate)
    
    return Out


class rnn_encoder(object):
    def __init__(self, img_size, hg_depth, dim_sentence, max_step_sentence, dim_rnn_cell, dr_rate):
        self.img_size = img_size
        self.hg_depth = hg_depth
        self.dim_sentence = dim_sentence
        self.max_step_sentence = max_step_sentence
        self.dim_rnn_cell = dim_rnn_cell
        self.dr_rate = dr_rate
        
        with tf.variable_scope("HGN/hg/word_weights"):                                              
            for i in range(hg_depth):
                exec('self.W_o%d = tf.get_variable('"'W_o%d'"', dtype=tf.float32,\
                                        initializer=tf.random_normal([self.dim_rnn_cell, \
                                                    int((self.img_size/(2**(self.hg_depth+1-%d)))**2)], \
                                                    stddev=stddev))' % (i+1, i+1, i+1))
                exec('self.b_o%d = tf.get_variable('"'b_o%d'"', dtype=tf.float32,\
                                       initializer=tf.random_normal([int((self.img_size/(2**(self.hg_depth+1-%d)))**2)], \
                                                                     stddev=stddev))' % (i+1, i+1, i+1))
    def encode(self, _x, _sen_len):
        _x_split = tf.transpose(_x, [2, 0, 1])
        _x_split = tf.reshape(_x_split, [-1, self.dim_sentence])
        _x_split = tf.split(_x_split, self.max_step_sentence, axis=0)

        with vs.variable_scope("HGN/rnn"):
            _rnn_cell = tf.nn.rnn_cell.BasicLSTMCell(self.dim_rnn_cell)
            _rnn_cell = tf.nn.rnn_cell.DropoutWrapper(_rnn_cell, output_keep_prob=1-self.dr_rate, input_keep_prob=1-self.dr_rate, state_keep_prob=1-self.dr_rate)
            _output, _state = tf.nn.static_rnn(_rnn_cell, _x_split, dtype=tf.float32, sequence_length=tf.squeeze(_sen_len))

        return _state[-1]

In [ ]:
import tensorflow as tf

def convBlock(curr_input, numIn, numOut, dr_rate):
    net = tf.layers.batch_normalization(curr_input)
    net = tf.nn.relu(net)
    net = tf.layers.conv2d(net, int(numOut/2), 1, padding='Same')
    net = tf.layers.dropout(net, rate=dr_rate)
    net = tf.layers.batch_normalization(net)
    net = tf.nn.relu(net)
    net = tf.layers.conv2d(net, int(numOut/2), 3, padding='Same')
    net = tf.layers.dropout(net, rate=dr_rate)
    net = tf.layers.batch_normalization(net)
    net = tf.nn.relu(net)
    net = tf.layers.conv2d(net, numOut, 1, padding='Same')
    net = tf.layers.dropout(net, rate=dr_rate)

    return net

def skipLayer(curr_input, numIn, numOut, dr_rate):
    if numIn == numOut:
        return curr_input
    else:
        net = tf.layers.conv2d(curr_input, numOut, 1, padding='Same')
        net = tf.layers.dropout(net, rate=dr_rate)
        return net

def Residual(curr_input, numIn, numOut, dr_rate):
    curr_convBlock = convBlock(curr_input, numIn, numOut, dr_rate)
    curr_skipLayer = skipLayer(curr_input, numIn, numOut, dr_rate)
    return curr_convBlock + curr_skipLayer

In [ ]:
import tensorflow as tf
import numpy as np
import random
import time
from hourglass_with_rnn import createModel

def train(img_resize, heatmap_resize, dim_sentence, max_step_sentence,
          batch_size, max_epoch, num_train, save_stride,
          num_hg_Depth, dim_hg_feat, dim_rnn_cell,
          restore_flag, restore_path, restore_epoch,
          total_images, total_heatmaps,
          train_text_inputs, train_sen_len, train_img_idx, train_pos_outputs,
          learning_rate, model_save_path):
    
    num_batch = num_train // batch_size
    
    ph_image = tf.placeholder(dtype=tf.float32, shape=[None, img_resize, img_resize, 3])
    ph_sen = tf.placeholder(dtype=tf.float32, shape=[None, dim_sentence, max_step_sentence])
    ph_sen_len = tf.placeholder(tf.int32, [None, 1])
    ph_heatmap = tf.placeholder(dtype=tf.float32, shape=[None, heatmap_resize, heatmap_resize, 1])
    ph_dropout = tf.placeholder(tf.float32)

    result_heatmap = createModel(curr_img = ph_image,
                                 curr_sen = ph_sen,
                                 curr_sen_len = ph_sen_len,
                                 img_size=heatmap_resize, 
                                 dim_sentence=dim_sentence,
                                 max_step_sentence=max_step_sentence, 
                                 num_hg_Depth=num_hg_Depth,
                                 dim_hg_feat=dim_hg_feat,
                                 dim_rnn_cell=dim_rnn_cell,
                                 dim_output=1,
                                 dr_rate=ph_dropout)
    loss = tf.reduce_mean((result_heatmap-ph_heatmap)**2)
    
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

    init = tf.global_variables_initializer()

    saver = tf.train.Saver()
    
    config = tf.ConfigProto()
    config.allow_soft_placement = True
    config.gpu_options.allow_growth = True

    print('Now ready to start the session.')
    
    with tf.Session(config=config) as sess:
        sess.run(init)

        if restore_flag == 1:
            saver.restore(sess, restore_path)

        for _epoch in range(max_epoch-restore_epoch):
            random.seed(_epoch)
            batch_shuffle = [i for i in range(num_train)]
            random.shuffle(batch_shuffle)

            total_train_loss = 0.0
            
            epoch_start_time = time.time()
            for i in range(num_batch):
                batch_idx = [batch_shuffle[idx] 
                             for idx in range(i * batch_size, (i + 1) * batch_size)]
                batch_inputs = train_text_inputs[batch_idx, :, :]
                batch_seq_len = train_sen_len[batch_idx, :]
                batch_img_idx = train_img_idx[batch_idx, :]
                batch_pos_output = train_pos_outputs[batch_idx, :]

                batch_images = np.zeros((batch_size, img_resize, img_resize, 3))
                batch_heatmaps = np.zeros((batch_size, heatmap_resize, heatmap_resize, 1))

                for ii in range(len(batch_idx)):
                    tmp_img = total_images['%04d' % batch_img_idx[ii, 0]]

                    batch_images[ii, :, :, :] = tmp_img        

                    tmp_heatmap = total_heatmaps['%04d_%03d_%03d' 
                                                 % (batch_img_idx[ii, 0], 
                                                    batch_pos_output[ii, 0], 
                                                    batch_pos_output[ii, 1])]  

                    batch_heatmaps[ii, :, :, 0] = tmp_heatmap

                train_feed_dict = {ph_sen: batch_inputs, ph_sen_len: batch_seq_len,
                                   ph_image: batch_images, ph_heatmap: batch_heatmaps,
                                   ph_dropout: 0.0}

                sess.run(optimizer, feed_dict=train_feed_dict)
                curr_train_loss = sess.run(loss, feed_dict=train_feed_dict)
                total_train_loss += curr_train_loss

                batch_end_time = time.time()
                total_time = batch_end_time - epoch_start_time 
                if i % 100 == 0:
                    print("batch loss : %s -> about %0.3f second left to finish this epoch" 
                          % (curr_train_loss, (total_time/(i+1))*(num_batch-i) ))

            total_train_loss = total_train_loss / num_batch

            print('current epoch : ' + str(_epoch+1+restore_epoch), 
                  ', current train loss : ' + str(total_train_loss))

            if (_epoch+1+restore_epoch) % save_stride == 0:
                model_saved_path = saver.save(sess, model_save_path)
                print("Model saved in file : %s" % model_saved_path)

In [ ]:
import tensorflow as tf
from tensorflow.python.ops import variable_scope as vs

stddev = 0.01

def hourglass_with_rnn(curr_input, rnn_feat, numDepth, numIn, numOut, dr_rate):
    up1 = curr_input
    up1 = Residual(up1, numIn, numOut, dr_rate)

    low1 = tf.layers.max_pooling2d(curr_input, 2, 2)
    low1 = Residual(low1, numIn, numOut, dr_rate)
    
    if numDepth > 1:
        low2 = hourglass_with_rnn(low1, rnn_feat, numDepth-1, numIn, numOut, dr_rate)
    else:
        low2 = Residual(low1, numIn, numOut, dr_rate)
    
    low3 = Residual(low2, numIn, numOut-1, dr_rate)
    
    with tf.variable_scope("word_weights", reuse=True):
        if numDepth == 4:
            _W_o4 = tf.get_variable('W_o4')
            _b_o4 = tf.get_variable('b_o4')
        elif numDepth == 3:
            _W_o3 = tf.get_variable('W_o3')
            _b_o3 = tf.get_variable('b_o3')
        elif numDepth == 2:
            _W_o2 = tf.get_variable('W_o2')
            _b_o2 = tf.get_variable('b_o2')
        elif numDepth == 1:
            _W_o1 = tf.get_variable('W_o1')
            _b_o1 = tf.get_variable('b_o1')
        else:
            raise ValueError('Code need to be fixed due to changed numDepth')
    
    if numDepth == 4:
        fitted_rnn = tf.matmul(rnn_feat, _W_o4)+_b_o4
    elif numDepth == 3:
        fitted_rnn = tf.matmul(rnn_feat, _W_o3)+_b_o3
    elif numDepth == 2:
        fitted_rnn = tf.matmul(rnn_feat, _W_o2)+_b_o2
    elif numDepth == 1:
        fitted_rnn = tf.matmul(rnn_feat, _W_o1)+_b_o1
    else:
        raise ValueError('Code need to be fixed due to changed numDepth')

    fitted_rnn = tf.nn.dropout(fitted_rnn, 1-dr_rate)
    
    fitted_rnn = tf.reshape(fitted_rnn, [tf.shape(rnn_feat)[0], 
                                        tf.to_int32(tf.sqrt(tf.to_float(tf.shape(fitted_rnn)[1]))),
                                        tf.to_int32(tf.sqrt(tf.to_float(tf.shape(fitted_rnn)[1])))])
    fitted_rnn = tf.expand_dims(fitted_rnn, axis=3)
    
    fuse_low3 = tf.concat([low3, fitted_rnn], axis=3)
        
    up2 = tf.image.resize_nearest_neighbor(fuse_low3, 2*tf.shape(fuse_low3)[1:3])

    return tf.add(up1, up2)


def lin(curr_input, numIn, numOut, dr_rate):
    l = tf.layers.conv2d(curr_input, numOut, 1, padding='Same')
    l = tf.layers.dropout(l, rate=dr_rate)
    l = tf.layers.batch_normalization(l)
    return tf.nn.relu(l)


def createModel(curr_img, curr_sen, curr_sen_len,
                img_size, dim_sentence, max_step_sentence, 
                num_hg_Depth, dim_hg_feat, dim_rnn_cell, dim_output,
                dr_rate):

    # image size must be 256 by 256.
    curr_rnn_encoder= rnn_encoder(img_size, num_hg_Depth, dim_sentence, max_step_sentence, dim_rnn_cell, dr_rate)
    
    rnn_feat = curr_rnn_encoder.encode(curr_sen, curr_sen_len)
    
    with vs.variable_scope('HGN'):
        with vs.variable_scope('pre'):
            cnv1 = tf.layers.conv2d(curr_img, filters=dim_hg_feat/4, kernel_size=7, strides=2, padding='Same')
            cnv1 = tf.layers.dropout(cnv1, rate=dr_rate)

            cnv1 = tf.layers.batch_normalization(cnv1)
            cnv1 = tf.nn.relu(cnv1)
        
        with vs.variable_scope('r1'):
            r1 = Residual(cnv1, dim_hg_feat/4, dim_hg_feat/2, dr_rate)

        pool = tf.layers.max_pooling2d(r1, 2, 2)

        with vs.variable_scope('r4'):    
            r4 = Residual(pool, dim_hg_feat/2, dim_hg_feat/2, dr_rate)

        with vs.variable_scope('r5'):    
            r5 = Residual(r4, dim_hg_feat/2, dim_hg_feat, dr_rate)    
    
        with vs.variable_scope('hg'):  
            hg = hourglass_with_rnn(r5, rnn_feat, num_hg_Depth, dim_hg_feat, dim_hg_feat, dr_rate)

        with vs.variable_scope('ll'):      
            ll = Residual(hg, dim_hg_feat, dim_hg_feat, dr_rate)
            ll = lin(ll, dim_hg_feat, dim_hg_feat, dr_rate)
            
        with vs.variable_scope('out'): 
            Out = tf.layers.conv2d(ll, filters=dim_output, kernel_size=1, strides=1, padding='Same')
            Out = tf.layers.dropout(Out, rate=dr_rate)
    
    return Out


class rnn_encoder(object):
    def __init__(self, img_size, hg_depth, dim_sentence, max_step_sentence, dim_rnn_cell, dr_rate):
        self.img_size = img_size
        self.hg_depth = hg_depth
        self.dim_sentence = dim_sentence
        self.max_step_sentence = max_step_sentence
        self.dim_rnn_cell = dim_rnn_cell
        self.dr_rate = dr_rate
        
        with tf.variable_scope("HGN/hg/word_weights"):                                              
            for i in range(hg_depth):
                exec('self.W_o%d = tf.get_variable('"'W_o%d'"', dtype=tf.float32,\
                                        initializer=tf.random_normal([self.dim_rnn_cell, \
                                                    int((self.img_size/(2**(self.hg_depth+1-%d)))**2)], \
                                                    stddev=stddev))' % (i+1, i+1, i+1))
                exec('self.b_o%d = tf.get_variable('"'b_o%d'"', dtype=tf.float32,\
                                       initializer=tf.random_normal([int((self.img_size/(2**(self.hg_depth+1-%d)))**2)], \
                                                                     stddev=stddev))' % (i+1, i+1, i+1))
    def encode(self, _x, _sen_len):
        _x_split = tf.transpose(_x, [2, 0, 1])
        _x_split = tf.reshape(_x_split, [-1, self.dim_sentence])
        _x_split = tf.split(_x_split, self.max_step_sentence, axis=0)

        with vs.variable_scope("HGN/rnn"):
            _rnn_cell = tf.nn.rnn_cell.BasicLSTMCell(self.dim_rnn_cell)
            _rnn_cell = tf.nn.rnn_cell.DropoutWrapper(_rnn_cell, output_keep_prob=1-self.dr_rate, input_keep_prob=1-self.dr_rate, state_keep_prob=1-self.dr_rate)
            _output, _state = tf.nn.static_rnn(_rnn_cell, _x_split, dtype=tf.float32, sequence_length=tf.squeeze(_sen_len))

        return _state[-1]

In [ ]:
restore_flag = 0
restore_epoch = 0

tf.reset_default_graph()

train(img_resize = 256, # must be fixed
      heatmap_resize = 64, # must be fixed
      dim_sentence = dim_sentence,
      max_step_sentence = max_step_sentence,
      batch_size = 8, 
      max_epoch = 150,
      num_train = num_train,
      save_stride = 1,
      num_hg_Depth = 4, # good be fixed
      dim_hg_feat = 256, # good be fixed
      dim_rnn_cell = 256, # good be fixed
      restore_flag = 0,
      restore_path = '/content/drive/MyDrive/machine_listening_2020/Project/train_model/model_.ckpt',
      restore_epoch = 0,
      total_images = total_images,
      total_heatmaps = total_heatmaps,
      train_text_inputs = train_text_inputs,
      train_sen_len = train_sen_len,
      train_img_idx = train_img_idx,
      train_pos_outputs = train_pos_outputs,
      learning_rate = 0.00001, # good be fixed
      model_save_path = '/content/drive/MyDrive/machine_listening_2020/Project/train_model/model_.ckpt') # good be fixed

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
Instructions for updating:
Use keras.layers.MaxPooling2